# 02 - IOI Circuit Discovery with EAP

In [ ]:
# Ensure the repo root (containing `src/` and `data/`) is on sys.path
import sys, os
from pathlib import Path

def add_repo_root(marker_dir='src'):
    path = os.path.abspath(os.getcwd())
    prev = None
    while path and path != prev:
        if os.path.isdir(os.path.join(path, marker_dir)):
            if path not in sys.path:
                sys.path.insert(0, path)
            return path
        prev = path
        path = os.path.dirname(path)
    nb_dir = os.path.abspath(os.getcwd())
    candidate = os.path.abspath(os.path.join(nb_dir, '..'))
    if os.path.isdir(os.path.join(candidate, marker_dir)):
        if candidate not in sys.path:
            sys.path.insert(0, candidate)
        return candidate
    return os.path.abspath(os.getcwd())

REPO_ROOT = add_repo_root()
DATA_PATH = str(Path(REPO_ROOT) / 'data' / 'ioi_examples.json')


In [ ]:
import torch
from transformer_lens import HookedTransformer
from src.ioi_task import load_ioi_examples
from src.circuit_finder import run_eap_on_ioi
from src.visualization import plot_head_importance, plot_top_heads_graph

model = HookedTransformer.from_pretrained('gpt2-small')
examples = load_ioi_examples(DATA_PATH)
clean_ids = []
corr_ids = []
for ex in examples:
    clean_ids.append(model.to_tokens(ex.clean)[0])
    corr_ids.append(model.to_tokens(ex.corrupted)[0])
clean = torch.nn.utils.rnn.pad_sequence(clean_ids, batch_first=True, padding_value=model.tokenizer.eos_token_id)
corr = torch.nn.utils.rnn.pad_sequence(corr_ids, batch_first=True, padding_value=model.tokenizer.eos_token_id)

res = run_eap_on_ioi(model, clean, corr, target_token_idx=-1)
plot_head_importance(res.head_importance, save_path='results/plots/head_importance_ioi.png')
plot_top_heads_graph(res.head_importance, save_path='results/plots/top_heads_ioi.png')
res.baseline_delta, res.head_importance.shape
